In [1]:
import pandas as pd
import os
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report
from sklearn.metrics import accuracy_score
from sklearn.metrics import precision_score, recall_score, f1_score, classification_report
v

In [2]:
# Load tokenizer first
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

In [3]:
from google.colab import files
uploaded = files.upload()  # Select your enron.zip file

Saving enron1.zip to enron1.zip


In [5]:
!unzip enron1.zip -d /content/enron/

Archive:  enron1.zip
replace /content/enron/__MACOSX/._enron? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [6]:
enron_path = "/content/enron/enron"

In [7]:
# obtain data from enron zip file
def extract_data(path):
    emails = []
    labels = []
    for i in range(1, 7):
        enron_path = os.path.join(path, f"enron{i}")
        spam_path = os.path.join(enron_path,"spam")
        ham_path = os.path.join(enron_path,"ham")
        print(spam_path)
        for filename in os.listdir(spam_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(spam_path, filename)

                with open(file_path, 'r', encoding='latin-1') as f:
                    txt = f.read()
                    emails.append(txt)
                    labels.append('spam')

        for filename in os.listdir(ham_path):
            if filename.endswith('.txt'):
                file_path = os.path.join(ham_path, filename)

                with open(file_path, 'r', encoding='latin-1') as f:
                    txt = f.read()
                    emails.append(txt)
                    labels.append('ham')

    return emails, labels

In [8]:
emails, spam_labels = extract_data(enron_path)
print(len(emails))
print(len(spam_labels))

/content/enron/enron/enron1/spam
/content/enron/enron/enron2/spam
/content/enron/enron/enron3/spam
/content/enron/enron/enron4/spam
/content/enron/enron/enron5/spam
/content/enron/enron/enron6/spam
33715
33715


In [9]:
spam_labels = [1 if label == "spam" else 0 for label in spam_labels]

In [13]:
from nltk.stem import WordNetLemmatizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import re
import nltk
nltk.download('punkt')
nltk.download('punkt_tab')
nltk.download('wordnet')
nltk.download('stopwords')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package punkt_tab to /root/nltk_data...
[nltk_data]   Package punkt_tab is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [14]:
# !pip install --upgrade --force-reinstall nltk
# import nltk
# nltk.download('punkt')
# nltk.download('punkt_tab') # Download the missing punkt_tab data

# preprocess the emails
def pre_process(text):

    stopwords = nltk.corpus.stopwords.words('english')

    lemmatizer = WordNetLemmatizer()

    text = text.lower()

    text = re.sub(r'[^a-zA-Z\s]', '', text)

    text = word_tokenize(text)

    final_text = [lemmatizer.lemmatize(word) for word in text if word not in stopwords]

    return final_text


In [15]:
preprocessed_emails = [pre_process(email) for email in emails]

In [16]:
enron_df = pd.DataFrame({"Emails" : preprocessed_emails})

In [17]:
enron_df['Labels'] = spam_labels

In [18]:
enron_df.head(10)

,Emails,Labels
0,"[subject, contemporary, course, decrease, mass...",1
1,"[subject, funny, may, aplomb, angola, may, pri...",1
2,"[subject, get, college, degree, online]",1
3,"[subject, best, dealon, viagra, place, charge,...",1
4,"[subject, epson, inkjet, cartridge, save, inkj...",1
5,"[subject, roller, coaster, cigar, email, loadi...",1
6,"[subject, smart, way, control, spam, special, ...",1
7,[subject],1
8,"[subject, brand, name, generic, overexpenditur...",1
9,"[subject, discreet, cheapest, pr, escription, ...",1


In [19]:
train_txt, test_txt, train_label, test_label = train_test_split(emails, spam_labels,
                                                                test_size=0.2, random_state=42)


In [20]:
len(train_txt)

26972

In [21]:
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

In [22]:
train_encodings = tokenizer(
    train_txt,
    truncation=True,
    padding=True,
    max_length=128
)

test_encodings = tokenizer(
    test_txt,
    truncation=True,
    padding=True,
    max_length=128
)

In [25]:
import torch
from torch.utils.data import Dataset, DataLoader

class TextDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings  # Store tokenized inputs (input_ids, attention_mask)
        self.labels = labels        # Store labels

    def __getitem__(self, idx):
        # Fetch tokenized data for this index
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item["labels"] = torch.tensor(self.labels[idx])  # Add label
        return item

    def __len__(self):
        return len(self.labels)  # Total number of samples


In [26]:
train_dataset = TextDataset(train_encodings, train_label)

train_loader = DataLoader(
    train_dataset,
    batch_size=16,
    shuffle=True
)

test_dataset = TextDataset(test_encodings, test_label)

test_loader = DataLoader(
    test_dataset,
    batch_size=16
)

In [27]:
from transformers import AutoModelForSequenceClassification

In [28]:
from torch.optim import AdamW


In [29]:
num_labels = 2  # number of classes
batch_size = 16
learning_rate = 2e-5  # Standard for BERT fine-tuning
epochs = 3

# initialize the model for classification
model = AutoModelForSequenceClassification.from_pretrained("bert-base-uncased", num_labels=2)

# # move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

# initialize the type of optimizer
optimizer = AdamW(model.parameters(), lr=learning_rate)

model.train()  # Set model to training mode

for epoch in range(epochs):
    total_loss = 0

    for batch in train_loader:
        # Move batch to GPU
        input_ids = batch['input_ids'].to(device)
        attention_mask = batch['attention_mask'].to(device)
        labels = batch['labels'].to(device)

        # Forward pass
        outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
        loss = outputs.loss

        # Backward pass
        loss.backward()
        optimizer.step()
        optimizer.zero_grad()

        total_loss += loss.item()

    print(f"Epoch {epoch+1} | Avg Loss: {total_loss/len(train_loader):.2f}")

model.safetensors:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at bert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Epoch 1 | Avg Loss: 0.05
Epoch 2 | Avg Loss: 0.01
Epoch 3 | Avg Loss: 0.01


In [31]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

def evaluate(model, val_loader):
    model.eval()
    val_loss = 0
    predictions = []
    true_labels = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids = batch['input_ids'].to(device)
            attention_mask = batch['attention_mask'].to(device)
            labels = batch['labels'].to(device)

            outputs = model(input_ids, attention_mask=attention_mask, labels=labels)
            val_loss += outputs.loss.item()

            logits = outputs.logits
            preds = torch.argmax(logits, dim=1).cpu().numpy()
            predictions.extend(preds)
            true_labels.extend(labels.cpu().numpy())

    # Calculate metrics
    accuracy = accuracy_score(true_labels, predictions)
    precision = precision_score(true_labels, predictions, average='binary')  # For multi-class, use 'weighted' or 'macro'
    recall = recall_score(true_labels, predictions, average='binary')
    f1 = f1_score(true_labels, predictions, average='binary')
    avg_loss = val_loss / len(val_loader)

    return avg_loss, accuracy, precision, recall, f1

# Run evaluation and get all metrics
val_loss, val_acc, val_precision, val_recall, val_f1 = evaluate(model, test_loader)

print(f"Validation Loss: {val_loss:.2f}")
print(f"Accuracy: {val_acc:.2f}")
print(f"Precision: {val_precision:.2f}")
print(f"Recall: {val_recall:.2f}")
print(f"F1-Score: {val_f1:.2f}")

Validation Loss: 0.04
Accuracy: 0.99
Precision: 1.00
Recall: 0.98
F1-Score: 0.99
